# 关于estimator的基本使用方法（以mnist数据为例）

[视频来源](https://www.youtube.com/watch?v=BhQW2OLzx_c)

# 导入基本的库函数

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
tf.__version__

'1.12.0'

# 加载数据

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/mnist/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 获取最大值所在列号，实际这里表示的类别标签
np.argmax(data.train.labels,axis=1)

array([7, 3, 4, ..., 5, 6, 8], dtype=int64)

In [4]:
# 基本的维度信息
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size,img_size)
num_channels = 1
num_classes = 10

In [5]:
# 定义输入的部分，包含了numpy_input_fn()和pandas_input_fn()两种方式
train_data_x = data.train.images
train_data_y = np.argmax(data.train.labels,axis=1)
test_data_x = data.test.images
test_data_y = np.argmax(data.test.labels,axis=1)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':np.array(train_data_x)},y=np.array(train_data_y),num_epochs=None,num_threads=10,shuffle=True)
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':np.array(test_data_x)},y=np.array(test_data_y),num_epochs=None,num_threads=10,shuffle=True)

In [6]:
some_imgs = data.test.images[0:9]
some_imgs_labels = data.test.labels[0:9]

In [7]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x = {'x':some_imgs},num_epochs=1,shuffle=False)

In [8]:
some_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':np.array(some_imgs)},y=np.array(some_imgs_labels),num_epochs=None,num_threads=10,shuffle=True)

# 实例化estimator

In [9]:
feature_x = tf.feature_column.numeric_column('x',shape=img_shape)

In [10]:
feature_columns = [feature_x]

In [11]:
num_hidden_units = [256,128,64]

In [12]:
model = tf.estimator.DNNClassifier(feature_columns = feature_columns,
                                  hidden_units = num_hidden_units,
                                  activation_fn = tf.nn.relu,
                                  n_classes = num_classes,
                                  model_dir = './model_file/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model_file/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026E727015F8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# 训练模型

In [13]:
model.train(input_fn=train_input_fn,steps=50)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_file/model.ckpt-50
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 50 into ./model_file/model.ckpt.
INFO:tensorflow:loss = 43.514675, step = 51
INFO:tensorflow:Saving checkpoints for 100 into ./model_file/model.ckpt.
INFO:tensorflow:Loss for final step: 35.899544.


# 拿模型训练新的数据

In [14]:
print('真实类别标签',np.argmax(some_imgs_labels,axis=1))

真实类别标签 [7 2 1 0 4 1 4 9 5]


In [15]:
result = model.predict(input_fn=predict_input_fn)
cls = [p['classes'] for p in result]
pred = np.array(cls,dtype='int').squeeze()
pred

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_file/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


array([7, 2, 1, 0, 4, 1, 4, 9, 4])